<a href="https://colab.research.google.com/github/jmoon312/2022carevent/blob/main/CAR_Demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install shap # will use this later

#Step 1: Importing Scripts & Setting up Data

I've set up a sample dataset based on a project I have with Eric Condie, a colleague here at Georgia Tech. In the paper, we're exploring the extent to which social media captures information about the risk of firm failure not captured in an auditor's going concern opinion.

I've anonymized a sample of data that we use in our going concern model, which is what I'll use for this demonstration. The data is available on github, which can be loaded directly into Google Colab.

This block of code will load the data, print descriptive statistics, and define a list (xvars) which contains the predictor variables we'll use in our classifier.

In [ ]:
import pandas as pd, numpy as np
# Add code to import data, and then describe it, include "xvars"

df = pd.read_csv("https://raw.githubusercontent.com/jmoon312/2022carevent/main/GCO_Sample_Data_2022CAR.csv")

print(df.describe().transpose())
xvars = ['Tenure','FeeRatio','FirmAge','ZScore','Size','Loss','Leverage',
         'ChLeverage','OpCF','Finance','Invest','Returns','Beta','Volatility']


# Step 2: Load scikit-learn packages needed to train ML Model
For this execise, we're going to train an classifier to predict the likelihood a firm uses a Big 4 auditor. This is not what Eric and I do in our paper; we model failure with is a lot harder than whether or not a Big 4 auditor is used! You're welcome to take this same code and try to predict GCOs (also much harder). 

The scikit-learn package has many different options for classifiers (Naive Bayes, Random Forests, Boosted ensemble models, etc.). We're going to train a Decision Tree since it's pretty fast.

The code below will identify training and testing datasets. I'm doing the bare minimum here (10 iterations, 3-fold cross-validation). Ordinarily I use 5-folds and more iterations), but that takes additional time.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report

# set up some parameters to try #
params = {'criterion':['gini','entropy'],
          'max_depth':[10,50,100,None],
          'random_state':[123],
          'class_weight':[None,'balanced']}

dt = DecisionTreeClassifier() # Leave this as default
train,test = train_test_split(df,train_size=0.80, random_state=123)
# 3-fold CV, 10 combos (ordinarily do more)
rs = RandomizedSearchCV(dt,param_distributions=params,n_jobs=-1,cv=3,n_iter=10,
                        verbose=5) 
res = rs.fit(train[xvars],train['Big4'])

#Step 3: Evaluate the model fit
We'll quickly look at how the model performed and then move on to feature importance.

In [ ]:
print("Confusin matrix (Rows = True 0/1, Columns = Predicted 0/1):")
print(confusion_matrix(test['Big4'],rs.predict(test[xvars])))
print("\n\nscikit-learn's classification report, which gives information on accuracy, precision, recall, etc.")
print(classification_report(test['Big4'],rs.predict(test[xvars])))
print("\n\nThe search identified this as the best estimator:")
print(res.best_estimator_)

# Step 4: Permutation Importance
I'll illustrate two approaches to evaluating feature importance. The first, referred to as permutation importance, randomly shuffles each feature and evaluates how much model performance is impacted. 

Here's an example of implementation, which is largely based on the [scikit-learn documentation](https://scikit-learn.org/stable/modules/permutation_importance.html). For a custom implementation, [see here](https://github.com/azsom/ODSC-East-2022/blob/main/interpretability_in_ML.ipynb).

In [ ]:
from sklearn.inspection import permutation_importance
metrics = ['balanced_accuracy','precision_macro','recall_macro']
r = permutation_importance(rs, test[xvars], test['Big4'],n_repeats=30,
                           random_state=123,scoring=metrics)

# r is dictionary with an entry for each metric. Each value (of the dictionary 
# Contains three items: mean importance (by feature), 
# standard deviation of importance (by feature), and the individual deviations

# This loop will print each feature provided it is "significant", based on it 
# being more than 2 standard deviations above 0:
print("\n\n----------------------------------------------------------------------")
print("Here are the significant features, in order of importance, by metric:")
for metric in r:
    print(f"{metric}")
    rm = r[metric]
    for i in rm.importances_mean.argsort()[::-1]:
        if rm.importances_mean[i] - 2 * rm.importances_std[i] > 0:
            print(f"    {xvars[i]:<8}"
                  f"{rm.importances_mean[i]:.3f}"
                  f" +/- {rm.importances_std[i]:.3f}")
            
print("----------------------------------------------------------------------")

print("\n\n----------------------------------------------------------------------")
print("Here are the features that are not significant, in order of importance, by metric:")
for metric in r:
    print(f"{metric}")
    rm = r[metric]
    for i in rm.importances_mean.argsort()[::-1]:
        if rm.importances_mean[i] - 2 * rm.importances_std[i] <= 0:
            print(f"    {xvars[i]:<8}"
                  f"{rm.importances_mean[i]:.3f}"
                  f" +/- {rm.importances_std[i]:.3f}")            
print("----------------------------------------------------------------------")

You can see subtle differences by metric, but for the most part each suggests Size, Tenure, and Firm Age are most important. 

Plotting these values can also be informative and easier to process. We'll limit this to balanced accuracy ([credit for this approach](https://github.com/azsom/ODSC-East-2022/blob/main/interpretability_in_ML.ipynb)).

In [ ]:
import matplotlib.pyplot as plt
metric = 'balanced_accuracy'
impts = r[metric]
sorted_indcs = impts.importances_mean.argsort()
plt.figure(figsize=(8,6))
plt.boxplot(impts.importances[sorted_indcs].T,labels=np.array(xvars)[sorted_indcs],vert=False)
plt.title("Permutation Importances (test set)")
plt.xlabel(f'decrease in {metric}')
plt.tight_layout()
plt.show()

# Step 5: Evaluating Feature Importance with Shapley Values
One significant limitation to the previous analysis is that features are correlated, so permutating a feature doesn't necessarily remove its influence from the model. Note that volatility appears insignificant to the model based on the analysis above, but consider correlations between volatility and the features:


In [ ]:
print(train[xvars].corr()['Volatility']) 

Volatility is significantly associated with Size, which is highly important. So if Size were excluded from the model, would volatility be important? If one is purely focused on which variables drive model performance, this correlation issue may not be that important, but it's hard to conclude that volatility "doesn't matter" from this analysis alone.

SHAP values don't fully alleviate this issue, but they do provide more insight into when features may or may not matter. The estimates are based on a cooperative game equilibrium. In essence, in a cooperative game there exists an equilibrium where the outcome is the weighted contributions of each player, with some players adding more or less to a given outcome. This same logic is applied to a trained model, and each prediction is expressed as the sum of individual feature importances.

Credit for my (developing) understanding of the intuition behind and how to use SHAP values comes from [this illustration](https://github.com/azsom/ODSC-East-2022/blob/main/interpretability_in_ML.ipynb), which was developed by Professor Andras Zsom (Brown University).

The example I provide below was adapted for the Decision Tree model we use, and is based on [this post](https://medium.com/analytics-vidhya/shap-part-3-tree-shap-3af9bcd7cd9b).

Below, I first review attributes of the original data and then the structure of what the shap package provides. 

In [ ]:
import shap
print(type(rs.best_estimator_)) # need to extract the estimator to use with shap (the RandomizedSearchCV object is not supported)
mod = rs.best_estimator_ # This is the DecisionTree classifier
exp = shap.TreeExplainer(mod,data=None,model_output='raw',feature_names=xvars,
                         feature_perturbation='tree_path_dependent') # sets up "explainer" using the trained model
shap_values = exp.shap_values(test[xvars]) # extracts SHAP values for the holdout (test) sample
print(f'Shape of test data: {test[xvars].shape}') # simply the shape of the input matrix
print(f'Type of shap values: {type(shap_values)}. Length of the list: {len(shap_values)}.') # two classes
print(f'Shape of shap_values: {np.array(shap_values).shape}') # two elements, each with shape matching data matrix

With SHAPs, you can examine the influence for individual observations. This could be useful if you had some mechanism to partition observations. For instance, how important is size for firms in industry X?

Example:

In [ ]:
shap.initjs()
print(f"Recall the mean of Big4 is {train['Big4'].mean()}, which is the 'base value'")
print("Looking at Row 0:")
print(test.iloc[0])
print("Here are the feature importances for this prediction:")
shap.force_plot(exp.expected_value[1], shap_values[1][0], features=test[xvars].iloc[0,:])

You can also examine summary plots:

In [ ]:
shap.summary_plot(shap_values,features=test[xvars])

With two classes the color coding isn't relevant (they are exact inverses), but if you were predicting multiple classes it would be useful. Here's the same plot where we only predict Big4 (Class 1).

In [ ]:
shap.summary_plot(shap_values,features=test[xvars],class_inds=[1])

The last plot we'll generate combines some of the earlier information, plotting the distribution of importances by feature:

In [ ]:
shap.summary_plot(shap_values[1],features=test[xvars])